In [2]:
search_for = 137
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0049285888671875
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 354865621
type: [1, 1, 1, 1, 137] 137
cases of this type: 352275361
100000 20.02005932688565
200000 33.63046481581241
300000 33.12879490416521
400000 33.22447182871089
500000 32.951302928364214
600000 32.98312836258754
700000 34.578020645788804
800000 33.14918874823132
900000 32.51839025210375
1000000 32.74952156386732
1100000 32.51671572941404
1200000 32.48621541502199
1300000 16.036478535648858
1400000 24.036107721959063
1500000 32.391078416038326
1600000 32.11025330624661
1700000 32.34996019378409
1800000 32.066083031003565
1900000 32.736620800791
2000000 33.82959356068667
2100000 32.34795944542049
2200000 32.23125617869829
2300000 32.3591242390374
2400000 32.692196263261685
2500000 32.66833488517807
2600000 12.830358541632995
2700000 27.93181242533901
2800000 41.09046111887035
2900000 41.492457259088475
3000000 40.5746279748069

29700000 28.050997733531958
29800000 30.548682279722133
29900000 31.061641051708566
30000000 31.62720093638761
30100000 31.747276909218005
30200000 31.050575026188557
30300000 31.68472888211329
30400000 33.513909449001225
30500000 32.28449354584965
30600000 31.271817638961288
30700000 12.53705030536029
30800000 33.50312935720201
30900000 33.61801560820276
31000000 32.983690780076266
31100000 20.039770817004502
31200000 24.261670989584545
31300000 33.583146369736205
31400000 33.53091114699609
31500000 33.73445145696479
31600000 32.51161474323745
31700000 34.12148947343409
31800000 32.82534005886128
31900000 32.01088880124613
32000000 31.907129263670555
32100000 30.938059766293573
32200000 7.390692820612089
32300000 32.04380927759789
32400000 31.466229391010035
32500000 31.600551434655276
32600000 32.9555650031699
32700000 32.07864840749619
32800000 31.348362856318094
32900000 32.65651200019532
33000000 32.54839764152079
33100000 32.619627312664214
33200000 20.255165658842298
33300000 25

59600000 17.20129191737751
59700000 23.97907510917585
59800000 31.20339630971329
59900000 28.909158637830735
60000000 29.06682187035207
60100000 29.03413278454794
60200000 26.608546696566922
60300000 28.412636329471628
60400000 22.6351334099979
60500000 13.332826453211
60600000 26.490021246172194
60700000 28.84669823556436
60800000 26.597393925810074
60900000 29.367091157045703
61000000 28.044855777980942
61100000 29.811191800002767
61200000 30.58084670060782
61300000 13.585851023705098
61400000 28.020805229295824
61500000 30.609879723134696
61600000 30.97897848316448
61700000 30.300630890472192
61800000 31.988843400950493
61900000 31.74893836080573
62000000 29.95168801451939
62100000 32.406700358534444
62200000 13.592684445601027
62300000 28.041157313932214
62400000 29.010028903860146
62500000 30.303078667471954
62600000 31.238214950177674
62700000 27.324375361377317
62800000 29.05206214355899
62900000 27.000253937889
63000000 18.466031648150057
63100000 18.209483008760138
63200000 29

89400000 8.69032250137314
89500000 27.368148441281143
89600000 26.294269298666812
89700000 26.941886368438137
89800000 27.279957846074527
89900000 26.722616402673566
90000000 28.605204736351656
90100000 25.968392000866867
90200000 27.29963099547132
90300000 26.835077419374752
90400000 26.399908802342537
90500000 26.03054138674675
90600000 25.89519802470104
90700000 10.305901506771104
90800000 23.09757284263449
90900000 24.660515709456607
91000000 23.086105153408138
91100000 24.38067304728199
91200000 23.99126566222143
91300000 11.683898946147387
91400000 18.868156203877348
91500000 24.301761702535174
91600000 23.712766870619028
91700000 23.753810254902014
91800000 23.878600452187644
91900000 17.299661522399155
92000000 18.26664801872102
92100000 25.41964758277433
92200000 25.39441989183598
92300000 27.18081846890155
92400000 25.911079651228846
92500000 27.247403113109446
92600000 26.51085834703177
92700000 27.123495576055483
92800000 27.253481277424918
92900000 27.27474755438006
930000

118500000 23.482357171153833
118600000 23.610312583410074
118700000 23.774452706418774
118800000 22.37228231429405
118900000 21.947632728755973
119000000 21.434058783134162
119100000 21.43652927923081
119200000 6.611560206356209
119300000 21.67963438077938
119400000 22.028507606407587
119500000 20.98215916580268
119600000 7.328707474105452
119700000 19.741511518539557
119800000 21.064414570768417
119900000 21.563361759620868
120000000 11.643553301474505
120100000 18.187873199770888
120200000 21.30928542908917
120300000 21.091856100395297
120400000 21.89021026803949
120500000 22.014837893934626
120600000 22.796916167719843
120700000 23.285050904104956
120800000 22.617579219000394
120900000 23.13522397715459
121000000 23.20814918165598
121100000 22.81089708059673
121200000 22.75362876098038
121300000 22.472939847911334
121400000 21.292703992470372
121500000 21.210261339987124
121600000 20.607311934847324
121700000 21.386417933159503
121800000 6.470122911399093
121900000 20.58105490774729

...

233000000 11.560692983587337
233100000 9.225756516521562
233200000 5.636097784232812
233300000 11.180237919729112
233400000 11.249060543049804
233500000 11.705509787193472
233600000 11.540598322721365
233700000 12.117656157875683
233800000 12.254989913304474
233900000 12.190353170906338
234000000 12.362808205640736
234100000 11.832943612679315
234200000 11.874329255892832
234300000 11.957999119327077
234400000 11.669689459426058
234500000 10.785935611323426
234600000 10.89198475525561
234700000 10.5837887374769
234800000 10.830230758881939
234900000 3.8770764290570763
235000000 11.049415501298581
235100000 10.901167475216527
235200000 11.018770008222484
235300000 4.647855097117716
235400000 9.171427269472593
235500000 10.847926758085597
235600000 10.913049988470847
235700000 8.190243265412052
235800000 6.689823407384456
235900000 10.88889561644484
236000000 11.17886634023961
236100000 11.481262506077664
236200000 11.462268151805018
236300000 11.849945367155247
236400000 12.449650209232

262000000 9.179711643784952
262100000 9.202903361972675
262200000 9.6707174075953
262300000 8.785301303340368
262400000 9.27148835932103
262500000 9.166416625838425
262600000 9.213260862676426
262700000 8.977470369828003
262800000 8.88305505188981
262900000 7.6825629385425085
263000000 4.499448553049216
263100000 8.237007594574322
263200000 8.388386465919542
263300000 8.139075348839222
263400000 8.296239595134653
263500000 8.449083818092298
263600000 2.4330359788622107
263700000 8.031296367389443
263800000 8.624159412004813
263900000 8.479915508371851
264000000 8.384463926457038
264100000 8.4850909679426
264200000 5.75301250552552
264300000 5.855805424453719
264400000 9.076282309764624
264500000 8.93644530940959
264600000 9.257677266649313
264700000 9.127293608972801
264800000 9.474863001832766
264900000 9.57653190674872
265000000 9.173518608455874
265100000 8.801179766556896
265200000 9.03940597988526
265300000 8.746403628404519
265400000 8.577956376186469
265500000 4.453649336533579


291300000 6.123465098500395
291400000 6.154468139731552
291500000 5.955926799031909
291600000 5.947330116308465
291700000 5.844727265099325
291800000 6.010318132209049
291900000 1.1565471340750735
292000000 6.102064767305309
292100000 5.889163955560953
292200000 5.903320741921611
292300000 5.8614692244728746
292400000 6.1896035431880385
292500000 6.194144168790057
292600000 6.163304131353915
292700000 3.9310672589385915
292800000 4.645058277710856
292900000 6.449074263989728
293000000 6.61721630644563
293100000 6.5763413602109635
293200000 6.706367982722769
293300000 6.322025776766354
293400000 6.6140498117517055
293500000 6.394091063809254
293600000 2.1736968026016044
293700000 6.774078125827476
293800000 6.31888248383689
293900000 6.078966671125514
294000000 6.158764647395955
294100000 5.895569822788618
294200000 5.770353412306003
294300000 5.827960619552828
294400000 4.551955631142221
294500000 2.869212552263473
294600000 5.71122213613348
294700000 5.792929477930384
294800000 5.8685

320500000 3.555227444696941
320600000 3.5914391295708405
320700000 3.709991233312225
320800000 3.520697077109063
320900000 3.621114697178669
321000000 3.5890658942619025
321100000 3.487406784392792
321200000 1.922114946620412
321300000 2.784774451842452
321400000 3.6878781827094076
321500000 3.45075839157846
321600000 3.15085335297337
321700000 1.5052975678927958
321800000 3.3296084708252227
321900000 3.3252138517065464
322000000 3.4588860863024573
322100000 3.2492213767833396
322200000 3.161659240812242
322300000 3.358663614605149
322400000 3.2394661676778256
322500000 3.1752945215340027
322600000 3.0639489939169766
322700000 2.0579852075416754
322800000 1.94377557763652
322900000 2.9704611839668553
323000000 3.202792226277025
323100000 3.156307746012219
323200000 3.1963912959124148
323300000 3.1386719674841994
323400000 3.157631555157288
323500000 3.2813801932352424
323600000 3.235168009857379
323700000 3.1986694187517544
323800000 1.5470383175847948
323900000 2.8466607403241815
3240

349100000 0.5817249320981436
349200000 0.6197060574306945
349300000 0.6125985047719776
349400000 0.5840044031235185
349500000 0.5945495402336505
349600000 0.5869214795978666
349700000 0.2548776812491695
349800000 0.26922814880763757
349900000 0.44183079752899007
350000000 0.3910983550089962
350100000 0.4083017364558286
350200000 0.35799816824521025
350300000 0.38337888585539326
350400000 0.3689201573844519
350500000 0.3556971976300717
350600000 0.3494880106991854
350700000 0.34411341172270515
350800000 0.3279270828523159
350900000 0.33050698427882796
351000000 0.16121372847563822
351100000 0.22836629805084668
351200000 0.3053406254411274
351300000 0.30221639799453615
351400000 0.30575406077136996
351500000 0.2683425207066059
351600000 0.22515393402261005
351700000 0.28938488928439815
351800000 0.24484536546195412
351900000 0.16491243039431772
352000000 0.2216123427682976
352100000 0.22292343089664976
352200000 0.19177489082161123
best so far: 0
type: [1, 1, 1, 137, 1] 137
cases of this